In [1]:
import pandas as pd

import os

In [2]:
!pip install pyenchant

grpcio 1.11.0 has requirement protobuf>=3.5.0.post1, but you'll have protobuf 3.4.1 which is incompatible.


In [3]:
announs = os.listdir("Announcement")
announs

['0083HK.csv',
 '0683HK.csv',
 '0012HK.csv',
 '0001HK.csv',
 '0017HK.csv',
 '0020HK.csv',
 '0016HK.csv',
 '0432HK.csv',
 '0041HK.csv',
 '0054HK.csv',
 '1200HK.csv']

In [4]:
##########################################
# clean up the text in the data with regex
##########################################
import re
import string

def clean_text(text):
    #text = str(row['fulltext'])

    # Remove newline characters
    cleantext = text.replace('\r\n', ' ')

    # Convert HTML punctuation chaaracters
    cleantext = cleantext.replace(' www.', ' ')   
    cleantext = cleantext.replace('.com ', ' ')    
    cleantext = cleantext.replace('.', ' ')
    cleantext = cleantext.replace(',', ' ')
    cleantext = cleantext.replace('!', ' ')
    cleantext = cleantext.replace('$;', ' ')
    cleantext = cleantext.replace('_', ' ')
    cleantext = cleantext.replace(';', ' ')
    cleantext = cleantext.replace(')', ' ')
    cleantext = cleantext.replace('(', ' ')   
    cleantext = cleantext.replace('>', ' ')  
    cleantext = cleantext.replace('<', ' ')  
    cleantext = cleantext.replace('-', ' ')  #take away hyphen and collapse hyphenated words
    cleantext = cleantext.replace(' the ', ' ')  
    cleantext = cleantext.replace(' of ', ' ')   
    cleantext = cleantext.replace(' in ', ' ')  
    cleantext = cleantext.replace(' and ', ' ')  
    cleantext = cleantext.replace(' by ', ' ')  
    cleantext = cleantext.replace(' to ', ' ')  
    cleantext = cleantext.replace(' at ', ' ')  
    cleantext = cleantext.replace(' on ', ' ') 
    cleantext = cleantext.replace(' for ', ' ')  
    cleantext = cleantext.replace(' be ', ' ')   
    cleantext = cleantext.replace(' is ', ' ')    
    cleantext = cleantext.replace(' or ', ' ')   
    cleantext = cleantext.replace(' we ', ' ')   
    cleantext = cleantext.replace(' that ', ' ')   
    cleantext = cleantext.replace(' our ', ' ')   
    cleantext = cleantext.replace(' as ', ' ')        
    cleantext = cleantext.replace(' from ', ' ')   
    cleantext = cleantext.replace(' are ', ' ')   
    cleantext = cleantext.replace(' with ', ' ')   
    cleantext = cleantext.replace(' us ', ' ')   
    cleantext = cleantext.replace(' was ', ' ')        
    cleantext = cleantext.replace(' this ', ' ')   
    cleantext = cleantext.replace(' an ', ' ')        
    cleantext = cleantext.replace(' by ', ' ')   
    cleantext = cleantext.replace(' sr ', ' ')      
    cleantext = cleantext.replace(' it ', ' ')  
    cleantext = cleantext.replace(' s ', ' ')  
    cleantext = cleantext.replace(' st ', ' ') 
    cleantext = cleantext.replace(' nd ', ' ') 
    cleantext = cleantext.replace(' rd ', ' ') 
    cleantext = cleantext.replace(' th ', ' ')


    #remove non alpha characters and specific noise
    cleantext = re.sub(r'\d+', ' ',cleantext)
    cleantext = re.sub(r'^b',' ',cleantext)
    cleantext = re.sub(r'[^\w]',' ',cleantext)
    cleantext = re.sub(u'[\u4e00-\u9fff]+','',cleantext)
    cleantext = cleantext.replace('xc xs', ' ')  
    cleantext = cleantext.replace('xe xs', ' ')  
    cleantext = cleantext.replace('xc xS', ' ')  
    cleantext = cleantext.replace('xe xS', ' ')  
    cleantext = cleantext.replace('xc xa', ' ')  
    cleantext = cleantext.replace('xe xa', ' ')  
    cleantext = cleantext.replace(' xc xc x', ' ')  
    cleantext = cleantext.replace(' xc ', ' ')  
    cleantext = cleantext.replace(' xe ', ' ')  
    cleantext = cleantext.replace(' xs ', ' ')  
    cleantext = cleantext.replace(' xa ', ' ')  
    cleantext = cleantext.replace(' ct ', ' ')  
    cleantext = cleantext.replace(' x ', ' ')  
    cleantext = cleantext.replace(' non exclusive ', ' non-exclusive ') 
    cleantext = cleantext.replace(' u ', ' ')   
    cleantext = cleantext.replace(' s ', ' ')  
    
    #remove specific noise
    cleantext = cleantext.translate(str.maketrans({'‘':' ','’':' '}))
    cleantext = cleantext.translate(str.maketrans({',':' ',',':' '}))
    cleantext = cleantext.translate(str.maketrans({'[':' ',']':' '}))
    cleantext = cleantext.translate(str.maketrans({'"':' ','%':' '}))
    cleantext = cleantext.translate(str.maketrans({'^':' ','*':' '}))

    #remove punctuation
    punctpattern = re.compile('[%s]' % re.escape(string.punctuation))
    cleanttext = re.sub(punctpattern,'', cleantext)

    #remove single letter word
    cleantext = re.sub('(\\b[A-Za-z] \\b|\\b [A-Za-z]\\b)', '', cleantext) 

    # Remove extra spaces
    cleantext = re.sub('\s+', ' ', cleantext).strip()
    
    

    return cleantext.lower()

In [ ]:
def check_eng(s):
    import enchant
    d = enchant.Dict("en_US")
    lis=[]
    for i in s.split(" "):
        if d.check(i):
            lis.append(i)
    return " ".join(lis)

In [ ]:
for i in announs:
    df = pd.read_csv("Announcement/{}".format(i))
    df["text"] = df["text"].apply(str).apply(clean_text).apply(check_eng)
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df.sort_values(by="Date",inplace=True)
    df.to_csv("Announcement_clean/{}".format(i))

In [ ]:
df = pd.read_csv("Announcement_clean/{}".format("0001HK.csv"), index_col=1)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [ ]:
df.head()